# Sqlite

In [2]:
import sqlite3

In [3]:
# connect database
db_connection = sqlite3.connect('mydb.db')
cursor = db_connection.cursor()

In [4]:
cursor.execute('''
    CREATE TABLE IF NOT EXISTS sample_data (
        id INTEGER PRIMARY KEY,
        name TEXT,
        age INTEGER
    )
''')
cursor.execute("INSERT INTO sample_data (name, age) VALUES ('Alice', 25)")
cursor.execute("INSERT INTO sample_data (name, age) VALUES ('Bob', 30)")
cursor.execute("INSERT INTO sample_data (name, age) VALUES ('Charlie', 22)")

In [5]:
# Commit and close the database connection
db_connection.commit()
db_connection.close()

## Sqlalchemy

In [6]:
import pandas as pd
from sqlalchemy import create_engine, text, inspect

In [7]:
engine = create_engine('sqlite:///mydb.db')

## Inspector

In [8]:
inspector = inspect(engine)

In [9]:
schemas = inspector.get_schema_names()
schemas

['main']

In [10]:
schema = schemas[0]
table_names = inspector.get_table_names(schema=schema)
table_names

['sample_data']

In [11]:
table_name = table_names[0]
columns = inspector.get_columns(table_name, schema=schema)
columns

[{'name': 'id',
  'type': INTEGER(),
  'nullable': True,
  'default': None,
  'primary_key': 1},
 {'name': 'name',
  'type': TEXT(),
  'nullable': True,
  'default': None,
  'primary_key': 0},
 {'name': 'age',
  'type': INTEGER(),
  'nullable': True,
  'default': None,
  'primary_key': 0}]

## Create table

In [23]:
with engine.connect() as conn:
    conn.execute(text('''
        CREATE TABLE IF NOT EXISTS sample_data2 (
            id INTEGER PRIMARY KEY,
            name TEXT,
            age INTEGER
        )
    '''))

## Insert

In [28]:
sql = "INSERT INTO sample_data2 (name, age) VALUES ('Alice', 26)"

with engine.connect() as conn:
    conn.execute(text(sql))
    conn.commit()

## Select

In [31]:
with engine.connect() as conn:
    result = conn.execute(text("SELECT * FROM sample_data2 LIMIT 20"))
    for row in result:
        print(row)

(1, 'Alice', 30)
(2, 'Alice', 30)


## Update

In [30]:
sql = """
UPDATE sample_data2 
SET age = 30 
WHERE name = 'Alice'
"""

with engine.connect() as conn:
    conn.execute(text(sql))
    conn.commit()

## Delete

In [32]:
sql = """
DELETE FROM sample_data2 
WHERE id = 1
"""

with engine.connect() as conn:
    conn.execute(text(sql))
    conn.commit()

## Drop table

In [33]:
sql = """
DROP TABLE sample_data2
"""

with engine.connect() as conn:
    conn.execute(text(sql))

## Write data into database

In [12]:
url = "https://raw.githubusercontent.com/ywchiu/riii/master/data/house-prices.csv"
df = pd.read_csv(url)
df

,Home,Price,SqFt,Bedrooms,Bathrooms,Offers,Brick,Neighborhood
0,1,114300,1790,2,2,2,No,East
1,2,114200,2030,4,2,3,No,East
2,3,114800,1740,3,2,1,No,East
3,4,94700,1980,3,2,3,No,East
4,5,119800,2130,3,3,3,No,East
...,...,...,...,...,...,...,...,...
123,124,119700,1900,3,3,3,Yes,East
124,125,147900,2160,4,3,3,Yes,East
125,126,113500,2070,2,2,2,No,North
126,127,149900,2020,3,3,1,No,West


In [13]:
df.dtypes

Home             int64
Price            int64
SqFt             int64
Bedrooms         int64
Bathrooms        int64
Offers           int64
Brick           object
Neighborhood    object
dtype: object

In [15]:
# https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.to_sql.html
# if_exists = fail, replace, append
df.to_sql('house_prices', index=False, if_exists='replace', con=engine)

128

## Read data from database

In [17]:
# https://pandas.pydata.org/docs/reference/api/pandas.read_sql.html
query = text("SELECT * FROM house_prices LIMIT 20")
df = pd.read_sql(query, con=engine)
df

,Home,Price,SqFt,Bedrooms,Bathrooms,Offers,Brick,Neighborhood
0,1,114300,1790,2,2,2,No,East
1,2,114200,2030,4,2,3,No,East
2,3,114800,1740,3,2,1,No,East
3,4,94700,1980,3,2,3,No,East
4,5,119800,2130,3,3,3,No,East
5,6,114600,1780,3,2,2,No,North
6,7,151600,1830,3,3,3,Yes,West
7,8,150700,2160,4,2,2,No,West
8,9,119200,2110,4,2,3,No,East
9,10,104000,1730,3,3,3,No,East


# Basic SQL
## Data Definition Language (DDL)
```sql
CREATE DATABASE databasename;
```
```sql
DROP DATABASE databasename;
```
```sql
CREATE TABLE persons (
    person_id int,
    first_name varchar(255),
    last_name varchar(255),
    address varchar(255),
    city varchar(255)
);
```
```sql
ALTER TABLE persons
ADD email varchar(255);
```
```sql
ALTER TABLE persons
DROP COLUMN column_name;
```
```sql
ALTER TABLE persons
RENAME COLUMN person_id to pid;
```
```sql
DROP TABLE persons;
```

## Data Manipulate Language (DML)
```sql
INSERT INTO persons (person_id, first_name, last_name, address, city)
VALUES (1, 'w', 'k', '11/22 soi3', 'bangkok');
```
```sql
SELECT * FROM persons LIMIT 10;
```
```sql
UPDATE persons
SET address = '15/22 soi3'
WHERE pid = 1;
```
```sql
DELETE FROM persons
WHERE pid = 1;
```

# CockroachDB on cloud

- Get free db [https://cockroachlabs.cloud/](https://cockroachlabs.cloud/)
- Login with google account
- Create cluster
- Save password
- Connect
  - Language: Python
  - Select tool SQLAlchemy
  - Operating system: Linux

## Install lib

In [ ]:
! pip install sqlalchemy-cockroachdb

## Download CA Cert (Linux)

In [ ]:
! curl --create-dirs -o $HOME/.postgresql/root.crt 'https://cockroachlabs.cloud/clusters/...'

## Set DATABASE_URL

In [ ]:
import os
os.environ['DATABASE_URL'] = "cockroachdb://..."

## Test Connection

In [ ]:
import os
from sqlalchemy import create_engine, text

engine = create_engine(os.environ["DATABASE_URL"])
conn = engine.connect()

res = conn.execute(text("SELECT now()")).fetchall()
print(res)

[(datetime.datetime(2023, 10, 23, 8, 47, 42, 6654, tzinfo=datetime.timezone.utc),)]


## Extract Transform Load (ETL)

### Extract

In [ ]:
import pandas as pd
url = "https://raw.githubusercontent.com/ywchiu/riii/master/data/house-prices.csv"
df = pd.read_csv(url)
df

,Home,Price,SqFt,Bedrooms,Bathrooms,Offers,Brick,Neighborhood
0,1,114300,1790,2,2,2,No,East
1,2,114200,2030,4,2,3,No,East
2,3,114800,1740,3,2,1,No,East
3,4,94700,1980,3,2,3,No,East
4,5,119800,2130,3,3,3,No,East
...,...,...,...,...,...,...,...,...
123,124,119700,1900,3,3,3,Yes,East
124,125,147900,2160,4,3,3,Yes,East
125,126,113500,2070,2,2,2,No,North
126,127,149900,2020,3,3,1,No,West


### Transform

### Load

In [ ]:
df.to_sql('house_prices', index=False, if_exists='replace', con=engine)

128

## ETL pipeline

In [ ]:
# Extract
query = text("SELECT * FROM house_prices")
with engine.connect() as conn:
  df = pd.read_sql(query, con=conn)
df

,Home,Price,SqFt,Bedrooms,Bathrooms,Offers,Brick,Neighborhood
0,1,114300,1790,2,2,2,No,East
1,2,114200,2030,4,2,3,No,East
2,3,114800,1740,3,2,1,No,East
3,4,94700,1980,3,2,3,No,East
4,5,119800,2130,3,3,3,No,East
...,...,...,...,...,...,...,...,...
123,124,119700,1900,3,3,3,Yes,East
124,125,147900,2160,4,3,3,Yes,East
125,126,113500,2070,2,2,2,No,North
126,127,149900,2020,3,3,1,No,West


In [ ]:
# Transform

In [ ]:
# Load
df.to_sql('house_prices', index=False, if_exists='replace', con=engine)